In [10]:
#https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson
#https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

In [11]:
import os
import requests
import pandas as pd
from werkzeug.utils import secure_filename
from osgeo import ogr

In [12]:
url_feature_server = "https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer/"
layer_def = requests.get(url_feature_server + '/?f=pjson').json()

df = pd.DataFrame(layer_def["layers"])
df

,id,name,parentLayerId,defaultVisibility,subLayerIds,minScale,maxScale,type,geometryType
0,0,Boreholes,-1,False,None,0,0,Feature Layer,esriGeometryPoint
1,1,Crossings,-1,False,None,0,0,Feature Layer,esriGeometryPoint
2,2,Levee Stations,-1,False,None,0,0,Feature Layer,esriGeometryPoint
3,3,Piezometers,-1,False,None,0,0,Feature Layer,esriGeometryPoint
4,4,Pump Stations,-1,False,None,0,0,Feature Layer,esriGeometryPoint
5,5,Relief Wells,-1,False,None,0,0,Feature Layer,esriGeometryPoint
6,6,Alignment Lines,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
7,7,Closure Structures,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
8,8,Cross Sections,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
9,9,Embankments,-1,True,None,0,0,Feature Layer,esriGeometryPolyline


In [13]:
download_folder = os.path.abspath("./download")

In [14]:
!mkdir -p {download_folder}

In [15]:
id_list = [14]
#id_list = range(0, df.shape[0])


def featurelayer2shapefile_ogr(featurelayer_url, shapefile_path):

    
    #ogr2ogr -overwrite -f 'ESRI Shapefile' <Shapefile> <ArcServerFeatureServer>/<LayerID>/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC
    
    query_url = '{}/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC'.format(featurelayer_url)
    print(query_url)
    ds = ogr.Open(query_url)
    layer = ds.GetLayerByIndex(0)
    feature_count = layer.GetFeatureCount()
    print("Input Feature Count: {:,}".format(feature_count))
    driver_out = ogr.GetDriverByName("ESRI Shapefile")
    # Remove output shapefile if it already exists
    if os.path.exists(shapefile_path):
        driver_out.DeleteDataSource(shapefile_path)
    ds_out = driver_out.CreateDataSource(shapefile_path)
    layer_out = ds_out.CopyLayer(layer, "layer")
    print("Output Feature Count: {:,} at {}".format(layer_out.GetFeatureCount(), shapefile_path))
    ds = None
    ds_out = None
    

def row_func(row):
    name = row["name"]    
    id = row["id"]
    if id_list is not None and type(id_list) is list:
        if int(id) not in id_list:
            return
    name_safe = secure_filename(name)
    shp_path = os.path.join(download_folder, '{}.shp'.format(name_safe))
    print("-"*80)
    print("id: {}; name: {}; file: {}".format(id, name, shp_path))
    featurelayer_url = "{}/{}/".format(url_feature_server, id)
    featurelayer2shapefile_ogr(featurelayer_url, shp_path)

In [16]:
%%time
# the bulk download normally takes ~30 mins
_ = df.apply(row_func, axis=1)

--------------------------------------------------------------------------------
id: 14; name: Leveed Areas; file: /home/jovyan/work/National_Levee_Database/download/Leveed_Areas.shp
https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer//14//query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC
Input Feature Count: 7,146


Warning 6: Normalized/laundered field name: 'LEVEE_STATION_CODE' to 'LEVEE_STAT'
Warning 6: Normalized/laundered field name: 'FEATURE_NAME' to 'FEATURE_NA'
Warning 6: Normalized/laundered field name: 'MIN_OVERTOP_EVENT' to 'MIN_OVERTO'
Warning 6: Normalized/laundered field name: 'EGRESS_NUMBER' to 'EGRESS_NUM'
Warning 6: Normalized/laundered field name: 'COMPUTED_SOURCE_DATE' to 'COMPUTED_S'
Warning 6: Normalized/laundered field name: 'Shape__Area' to 'Shape__Are'
Warning 6: Normalized/laundered field name: 'Shape__Length' to 'Shape__Len'
Warning 6: Normalized/laundered field name: 'COMPUTED_SOURCE' to 'COMPUTED_1'
Warning 6: Normalized/laundered field name: 'LEVEED_AREA_SOURCE' to 'LEVEED_ARE'
Warning 6: Normalized/laundered field name: 'WARNING_INDICATOR' to 'WARNING_IN'
Warning 6: Normalized/laundered field name: 'EVACUATION_PLAN_INDICATOR' to 'EVACUATION'
Warning 6: Normalized/laundered field name: 'AREA_SQ_MILES' to 'AREA_SQ_MI'
Warning 6: Normalized/laundered field name: 'SYSTEM_

Output Feature Count: 7,146 at /home/jovyan/work/National_Levee_Database/download/Leveed_Areas.shp
CPU times: user 17.3 s, sys: 2.04 s, total: 19.3 s
Wall time: 2min 27s


In [17]:
!du {download_folder} -h

4.0K	/home/jovyan/work/National_Levee_Database/download/.ipynb_checkpoints
4.0K	/home/jovyan/work/National_Levee_Database/download/Boreholes
506M	/home/jovyan/work/National_Levee_Database/download


In [18]:
!ls {download_folder} -alh

total 506M
drwxr-xr-x 4 jovyan users 4.0K Mar 11 22:20 .
drwxr-xr-x 8 jovyan users 4.0K Mar 11 22:22 ..
drwxr-xr-x 2 jovyan users 4.0K Mar 10 19:47 Boreholes
-rw-r--r-- 1 jovyan users  32M Mar 10 18:39 Boreholes.dbf
-rw-r--r-- 1 jovyan users  145 Mar 10 18:38 Boreholes.prj
-rw-r--r-- 1 jovyan users 1.8M Mar 10 18:39 Boreholes.shp
-rw-r--r-- 1 jovyan users 396K Mar 10 18:39 Boreholes.shx
-rw-r--r-- 1 jovyan users  89M Mar 10 18:41 Crossings.dbf
-rw-r--r-- 1 jovyan users  145 Mar 10 18:39 Crossings.prj
-rw-r--r-- 1 jovyan users 3.9M Mar 10 18:41 Crossings.shp
-rw-r--r-- 1 jovyan users 887K Mar 10 18:41 Crossings.shx
drwxr-xr-x 2 jovyan users 4.0K Mar 11 22:20 .ipynb_checkpoints
-rw-r--r-- 1 jovyan users  32M Mar 11 22:22 Leveed_Areas.dbf
-rw-r--r-- 1 jovyan users  145 Mar 11 22:20 Leveed_Areas.prj
-rw-r--r-- 1 jovyan users 158M Mar 11 22:22 Leveed_Areas.shp
-rw-r--r-- 1 jovyan users  56K Mar 11 22:22 Leveed_Areas.shx
-rw-r--r-- 1 jovyan users 158M Mar 11 22:11 .nfs0000000006200119000000b

## References

https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson

https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html#get-wfs-layers-and-iterate-over-features

https://gdal.org/python/

https://gdal.org/drivers/vector/geojson.html#vector-geojson

https://gdal.org/drivers/vector/esrijson.html#vector-esrijson